#  Report:

## I selected columns "ki_value","ki_chance","ki_activities" are effective in prediction


In [1]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.metrics import mean_squared_error, r2_score
from sklearn import linear_model
from sklearn.preprocessing import LabelEncoder
from sklearn import preprocessing
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import LSTM
from keras.callbacks import EarlyStopping
from keras.layers import Dense,LSTM,GRU, Dropout, Flatten
from keras import optimizers
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
from sklearn.metrics import accuracy_score

import scipy.stats as stats

Using TensorFlow backend.


In [2]:
 
df = pd.read_csv('ki_dealdata.csv',sep=";")
 

selected_col=["ki_value","ki_chance","ki_activities"]
df.head(20)

,ki_deal,ki_company,ki_owner,ki_id,ki_value,ki_chance,ki_closestatus,ki_allblocks_closestatus,ki_deadline,ki_activities,ki_blocknumber,ki_created
0,20,6,6,1,105000,33,Offen,Verloren,2018-05-31,43,1,2018-03-28
1,20,6,6,2,105000,33,Offen,Verloren,2018-05-31,43,0,2018-03-28
2,20,6,6,203,105000,33,Offen,Verloren,2018-05-31,3,2,2018-03-28
3,20,6,6,356,105000,33,Offen,Verloren,2018-05-31,17,3,2018-03-28
4,20,6,6,513,105000,33,Offen,Verloren,2018-05-31,5,4,2018-03-28
5,20,6,6,664,105000,33,Offen,Verloren,2018-05-31,1,5,2018-03-28
6,20,6,6,806,105000,33,Offen,Verloren,2018-05-31,2,6,2018-03-28
7,20,6,6,955,105000,33,Offen,Verloren,2018-05-31,2,7,2018-03-28
8,20,6,6,1112,105000,33,Offen,Verloren,2018-05-31,2,8,2018-03-28
9,20,6,6,1305,105000,33,Offen,Verloren,2018-05-31,0,9,2018-03-28


In [3]:
uniq_deal=[]
for i in pd.Series(df.ki_deal).unique():
    uniq_deal.append(i)
print(uniq_deal)
 

[20, 24, 32, 34, 35, 40, 42, 45, 46, 48, 51, 52, 54, 56, 57, 59, 60, 62, 64, 68, 72, 73, 74, 77, 83, 84, 85, 88, 91, 92, 93, 94, 96, 98, 99, 100, 102, 104, 105, 106, 108, 109, 110, 114, 116, 119, 120, 124, 126, 130, 132, 136, 137, 139, 141, 143, 145, 148, 149, 151, 152, 153, 156, 157, 158, 159, 163, 164, 170, 173, 175, 176, 178, 186, 187, 189, 192, 194, 195, 198, 199, 207, 208, 209, 210, 211, 213, 214, 216, 217, 218, 219, 223, 225, 228, 229, 234, 238, 239, 240, 241, 245, 246, 247, 248, 249, 251, 252, 253, 254, 258, 260, 261, 262, 263, 264, 265, 266, 270, 271, 273, 277, 285, 286, 287, 288, 289, 291, 293, 295, 299, 300, 303, 305, 308, 309, 310, 311, 313, 315, 316, 317, 318, 319, 320, 321, 324, 330, 331, 332, 333, 334, 335, 336, 337, 338, 342, 344, 345, 347, 348, 349, 352, 353, 360, 362, 363, 364, 365, 367, 368, 375, 378, 379, 381, 383, 390, 393, 394, 395, 396, 398, 404, 410, 411, 412, 415, 418, 419, 420, 422, 424, 427, 428, 430, 432, 435, 436, 437, 438, 439, 440, 441, 442, 443, 444, 446,

In [4]:
from collections import defaultdict
records = defaultdict(list)
 

for (index_label, row_series) in df.iterrows():
    for j in uniq_deal:
        if j==row_series.values[0]:
            records[j].append([row_series.values[4],row_series.values[5],row_series.values[9]])
            
            

In [5]:
dfa = pd.DataFrame.from_dict(records, orient='index')
 

dfa.head(5)

,0,1,2,3,4,5,6,7,8,9,...,53,54,55,56,57,58,59,60,61,62
20,"[105000, 33, 43]","[105000, 33, 43]","[105000, 33, 3]","[105000, 33, 17]","[105000, 33, 5]","[105000, 33, 1]","[105000, 33, 2]","[105000, 33, 2]","[105000, 33, 2]","[105000, 33, 0]",...,None,None,None,None,None,None,None,None,None,None
24,"[35420, 50, 32]","[35420, 50, 31]","[35420, 90, 11]","[35420, 90, 2]","[35420, 90, 2]","[35420, 90, 2]","[35420, 90, 4]","[35420, 90, 2]","[35420, 90, 1]","[35420, 90, 3]",...,None,None,None,None,None,None,None,None,None,None
32,"[95000, 90, 19]","[95000, 90, 19]","[95100, 90, 5]",None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
34,"[300000, 50, 26]","[300000, 50, 26]","[300000, 50, 11]","[300000, 50, 3]","[300000, 50, 3]","[300000, 50, 3]","[300000, 50, 1]","[300000, 50, 2]","[300000, 50, 14]","[300000, 50, 11]",...,None,None,None,None,None,None,None,None,None,None
35,"[60000, 30, 10]","[60000, 30, 10]","[60000, 30, 9]","[60000, 30, 9]","[60000, 30, 13]","[60000, 30, 3]","[60000, 30, 3]","[60000, 30, 7]","[60000, 30, 2]","[60000, 30, 0]",...,None,None,None,None,None,None,None,None,None,None


In [6]:
dfa=dfa.fillna(0)
 

In [7]:
for i in range(288):
    for j in range(63):
        if dfa.iloc[i,j]==0:
            dfa.iloc[i,j]=[0,0,0]
print('done')

done


In [8]:
dfa.head(5)

,0,1,2,3,4,5,6,7,8,9,...,53,54,55,56,57,58,59,60,61,62
20,"[105000, 33, 43]","[105000, 33, 43]","[105000, 33, 3]","[105000, 33, 17]","[105000, 33, 5]","[105000, 33, 1]","[105000, 33, 2]","[105000, 33, 2]","[105000, 33, 2]","[105000, 33, 0]",...,"[0, 0, 0]","[0, 0, 0]","[0, 0, 0]","[0, 0, 0]","[0, 0, 0]","[0, 0, 0]","[0, 0, 0]","[0, 0, 0]","[0, 0, 0]","[0, 0, 0]"
24,"[35420, 50, 32]","[35420, 50, 31]","[35420, 90, 11]","[35420, 90, 2]","[35420, 90, 2]","[35420, 90, 2]","[35420, 90, 4]","[35420, 90, 2]","[35420, 90, 1]","[35420, 90, 3]",...,"[0, 0, 0]","[0, 0, 0]","[0, 0, 0]","[0, 0, 0]","[0, 0, 0]","[0, 0, 0]","[0, 0, 0]","[0, 0, 0]","[0, 0, 0]","[0, 0, 0]"
32,"[95000, 90, 19]","[95000, 90, 19]","[95100, 90, 5]","[0, 0, 0]","[0, 0, 0]","[0, 0, 0]","[0, 0, 0]","[0, 0, 0]","[0, 0, 0]","[0, 0, 0]",...,"[0, 0, 0]","[0, 0, 0]","[0, 0, 0]","[0, 0, 0]","[0, 0, 0]","[0, 0, 0]","[0, 0, 0]","[0, 0, 0]","[0, 0, 0]","[0, 0, 0]"
34,"[300000, 50, 26]","[300000, 50, 26]","[300000, 50, 11]","[300000, 50, 3]","[300000, 50, 3]","[300000, 50, 3]","[300000, 50, 1]","[300000, 50, 2]","[300000, 50, 14]","[300000, 50, 11]",...,"[0, 0, 0]","[0, 0, 0]","[0, 0, 0]","[0, 0, 0]","[0, 0, 0]","[0, 0, 0]","[0, 0, 0]","[0, 0, 0]","[0, 0, 0]","[0, 0, 0]"
35,"[60000, 30, 10]","[60000, 30, 10]","[60000, 30, 9]","[60000, 30, 9]","[60000, 30, 13]","[60000, 30, 3]","[60000, 30, 3]","[60000, 30, 7]","[60000, 30, 2]","[60000, 30, 0]",...,"[0, 0, 0]","[0, 0, 0]","[0, 0, 0]","[0, 0, 0]","[0, 0, 0]","[0, 0, 0]","[0, 0, 0]","[0, 0, 0]","[0, 0, 0]","[0, 0, 0]"


In [9]:
d={}
for (index_label, row_series) in df.iterrows():
    for z in uniq_deal:
        if z==row_series.values[0]:
            d[z]=[row_series.values[7]]

In [10]:
labels=[]
lb_make = LabelEncoder()
labels.append(lb_make.fit_transform([d[w] for w in uniq_deal]))

C:\Users\Majid\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [11]:
dfa["label"]=[f for f in labels[0]]
 

In [13]:
dfa.head(5)

,0,1,2,3,4,5,6,7,8,9,...,54,55,56,57,58,59,60,61,62,label
20,"[105000, 33, 43]","[105000, 33, 43]","[105000, 33, 3]","[105000, 33, 17]","[105000, 33, 5]","[105000, 33, 1]","[105000, 33, 2]","[105000, 33, 2]","[105000, 33, 2]","[105000, 33, 0]",...,"[0, 0, 0]","[0, 0, 0]","[0, 0, 0]","[0, 0, 0]","[0, 0, 0]","[0, 0, 0]","[0, 0, 0]","[0, 0, 0]","[0, 0, 0]",1
24,"[35420, 50, 32]","[35420, 50, 31]","[35420, 90, 11]","[35420, 90, 2]","[35420, 90, 2]","[35420, 90, 2]","[35420, 90, 4]","[35420, 90, 2]","[35420, 90, 1]","[35420, 90, 3]",...,"[0, 0, 0]","[0, 0, 0]","[0, 0, 0]","[0, 0, 0]","[0, 0, 0]","[0, 0, 0]","[0, 0, 0]","[0, 0, 0]","[0, 0, 0]",1
32,"[95000, 90, 19]","[95000, 90, 19]","[95100, 90, 5]","[0, 0, 0]","[0, 0, 0]","[0, 0, 0]","[0, 0, 0]","[0, 0, 0]","[0, 0, 0]","[0, 0, 0]",...,"[0, 0, 0]","[0, 0, 0]","[0, 0, 0]","[0, 0, 0]","[0, 0, 0]","[0, 0, 0]","[0, 0, 0]","[0, 0, 0]","[0, 0, 0]",0
34,"[300000, 50, 26]","[300000, 50, 26]","[300000, 50, 11]","[300000, 50, 3]","[300000, 50, 3]","[300000, 50, 3]","[300000, 50, 1]","[300000, 50, 2]","[300000, 50, 14]","[300000, 50, 11]",...,"[0, 0, 0]","[0, 0, 0]","[0, 0, 0]","[0, 0, 0]","[0, 0, 0]","[0, 0, 0]","[0, 0, 0]","[0, 0, 0]","[0, 0, 0]",1
35,"[60000, 30, 10]","[60000, 30, 10]","[60000, 30, 9]","[60000, 30, 9]","[60000, 30, 13]","[60000, 30, 3]","[60000, 30, 3]","[60000, 30, 7]","[60000, 30, 2]","[60000, 30, 0]",...,"[0, 0, 0]","[0, 0, 0]","[0, 0, 0]","[0, 0, 0]","[0, 0, 0]","[0, 0, 0]","[0, 0, 0]","[0, 0, 0]","[0, 0, 0]",1


In [14]:
train_x=dfa.iloc[:,:63]

In [15]:
y=dfa["label"]

In [16]:
print(train_x.shape)
print(y.shape)

(288, 63)
(288,)


In [17]:
y=y.values


In [18]:
train_xx = np.array(train_x)

In [19]:
train_xx=train_xx.tolist()

In [20]:
#Padding the sequence with the values in last row to max length
from keras.preprocessing import sequence
final_seq=sequence.pad_sequences(train_xx, maxlen=63, padding='post', dtype='float', truncating='post')

In [21]:
print(final_seq)

[[[1.0500e+05 3.3000e+01 4.3000e+01]
  [1.0500e+05 3.3000e+01 4.3000e+01]
  [1.0500e+05 3.3000e+01 3.0000e+00]
  ...
  [0.0000e+00 0.0000e+00 0.0000e+00]
  [0.0000e+00 0.0000e+00 0.0000e+00]
  [0.0000e+00 0.0000e+00 0.0000e+00]]

 [[3.5420e+04 5.0000e+01 3.2000e+01]
  [3.5420e+04 5.0000e+01 3.1000e+01]
  [3.5420e+04 9.0000e+01 1.1000e+01]
  ...
  [0.0000e+00 0.0000e+00 0.0000e+00]
  [0.0000e+00 0.0000e+00 0.0000e+00]
  [0.0000e+00 0.0000e+00 0.0000e+00]]

 [[9.5000e+04 9.0000e+01 1.9000e+01]
  [9.5000e+04 9.0000e+01 1.9000e+01]
  [9.5100e+04 9.0000e+01 5.0000e+00]
  ...
  [0.0000e+00 0.0000e+00 0.0000e+00]
  [0.0000e+00 0.0000e+00 0.0000e+00]
  [0.0000e+00 0.0000e+00 0.0000e+00]]

 ...

 [[5.0000e+04 1.0000e+02 7.0000e+00]
  [5.0000e+04 1.0000e+02 7.0000e+00]
  [0.0000e+00 0.0000e+00 0.0000e+00]
  ...
  [0.0000e+00 0.0000e+00 0.0000e+00]
  [0.0000e+00 0.0000e+00 0.0000e+00]
  [0.0000e+00 0.0000e+00 0.0000e+00]]

 [[3.9896e+04 9.0000e+01 5.0000e+00]
  [3.9896e+04 9.0000e+01 3.0000e+00]


In [22]:
model = Sequential()
model.add(LSTM(256, input_shape=(63, 3)))
#model.add(Dense(128, activation='relu', input_dim=100))
#model.add(Dense(64, activation='relu'))

model.add(Dense(1, activation='sigmoid'))

Instructions for updating:
Colocations handled automatically by placer.


In [23]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 256)               266240    
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 257       
Total params: 266,497
Trainable params: 266,497
Non-trainable params: 0
_________________________________________________________________


In [24]:
adam = Adam(lr=0.001)
chk = ModelCheckpoint('best_model.pkl', monitor='val_acc', save_best_only=True, mode='max', verbose=1)
model.compile(loss='mse', optimizer=adam, metrics=['accuracy'])
model.fit(final_seq,y,epochs=10, batch_size=64, callbacks=[chk])

Instructions for updating:
Use tf.cast instead.
Epoch 1/10
288/288 [==============================] - 2s 8ms/step - loss: 0.2472 - acc: 0.6215
Epoch 2/10
 64/288 [=====>........................] - ETA: 0s - loss: 0.2260 - acc: 0.8281

C:\Users\Majid\Anaconda3\lib\site-packages\keras\callbacks.py:434: RuntimeWarning: Can save best model only with val_acc available, skipping.
  'skipping.' % (self.monitor), RuntimeWarning)


288/288 [==============================] - 1s 3ms/step - loss: 0.2197 - acc: 0.7535
Epoch 3/10
288/288 [==============================] - 1s 3ms/step - loss: 0.2262 - acc: 0.6111
Epoch 4/10
288/288 [==============================] - 1s 3ms/step - loss: 0.1976 - acc: 0.7535
Epoch 5/10
288/288 [==============================] - 1s 3ms/step - loss: 0.1972 - acc: 0.7535
Epoch 6/10
288/288 [==============================] - 1s 3ms/step - loss: 0.1882 - acc: 0.7639
Epoch 7/10
288/288 [==============================] - 1s 3ms/step - loss: 0.1881 - acc: 0.7708
Epoch 8/10
288/288 [==============================] - 1s 3ms/step - loss: 0.1822 - acc: 0.7708
Epoch 9/10
288/288 [==============================] - 1s 3ms/step - loss: 0.1792 - acc: 0.7708
Epoch 10/10
288/288 [==============================] - 1s 3ms/step - loss: 0.1776 - acc: 0.7708


In [25]:
uniq_count = df['ki_deal'].value_counts()
uniq_count[20]

11

In [26]:
def test_generator():
    test_x=np.zeros((288,63,3))
    i=-1
    ret=[]
    for j in uniq_deal:
        i+=1
        n=uniq_count[j]-1
        maxx=np.random.randint(0,n)
        ret.append(maxx)
        
        for t in range(maxx):
            test_x[i][t][:]=final_seq[i][t]
            
    #print (ret)
    return test_x
        
    


In [27]:
result_acc=[]
for i in range(10):
    test_x=test_generator()
    test_preds = model.predict_classes(test_x)
    result_acc.append(accuracy_score(test_preds,y))
    
np.mean(result_acc)

0.7680555555555556